<a href="https://colab.research.google.com/github/satyajitghana/ProjektDepth/blob/master/notebooks/01_DenseDepth_DatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Depth Project - Dataset Creation

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My\ Drive/DepthProject

/content/gdrive/My Drive/DepthProject


In [0]:
!ls -l

total 2234578
drwx------ 2 root root       4096 May  3 15:54 DenseDepth
drwx------ 2 root root       4096 May  3 22:30 depth_dataset_cleaned
-rw------- 1 root root    1706380 May  3 15:22 depth_dataset_cleaned_raw.zip
-rw------- 1 root root 2286492537 May  3 14:31 depth_dataset_cleaned.zip


## Note

I came up with the conclusion that writing the images directly to google drive is very slow, instead write them into a .zip file directly, which i did on my local machine, this notebook however shows the concept of how i had tried different methods

## This is the write the image to folder 

### **PROCEED WITH ATMOST CAUTION**

In [0]:
!rm -r depth_dataset_cleaned/

^C


### Count the number of processed folders, should be 100

In [0]:
!ls depth_dataset_cleaned/fg_bg/ | wc -l

100


## Count the Processed files on each folder, should be 4000 in each

In [0]:
! find ./depth_dataset_cleaned/fg_bg/ -type d  | awk '{print "echo -n \""$0"  \";ls -l "$0" | grep -v total | wc -l" }' | sh 

./depth_dataset_cleaned/fg_bg/  100
./depth_dataset_cleaned/fg_bg/bg_000  4412
./depth_dataset_cleaned/fg_bg/bg_001  4752
./depth_dataset_cleaned/fg_bg/bg_002  4000
./depth_dataset_cleaned/fg_bg/bg_003  4000
./depth_dataset_cleaned/fg_bg/bg_004  4000
./depth_dataset_cleaned/fg_bg/bg_005  4000
./depth_dataset_cleaned/fg_bg/bg_006  492
./depth_dataset_cleaned/fg_bg/bg_007  0
./depth_dataset_cleaned/fg_bg/bg_008  0
./depth_dataset_cleaned/fg_bg/bg_009  0
./depth_dataset_cleaned/fg_bg/bg_010  0
./depth_dataset_cleaned/fg_bg/bg_011  0
./depth_dataset_cleaned/fg_bg/bg_012  0
./depth_dataset_cleaned/fg_bg/bg_013  0
./depth_dataset_cleaned/fg_bg/bg_014  0
./depth_dataset_cleaned/fg_bg/bg_015  4000
./depth_dataset_cleaned/fg_bg/bg_016  4000
./depth_dataset_cleaned/fg_bg/bg_017  4000
./depth_dataset_cleaned/fg_bg/bg_018  4000
./depth_dataset_cleaned/fg_bg/bg_019  4000
./depth_dataset_cleaned/fg_bg/bg_020  4000
./depth_dataset_cleaned/fg_bg/bg_021  4000
./depth_dataset_cleaned/fg_bg/bg_022  4000


In [0]:
!ls depth_dataset_cleaned/fg_bg/

bg_000	bg_010	bg_020	bg_030	bg_040	bg_050	bg_060	bg_070	bg_080	bg_090
bg_001	bg_011	bg_021	bg_031	bg_041	bg_051	bg_061	bg_071	bg_081	bg_091
bg_002	bg_012	bg_022	bg_032	bg_042	bg_052	bg_062	bg_072	bg_082	bg_092
bg_003	bg_013	bg_023	bg_033	bg_043	bg_053	bg_063	bg_073	bg_083	bg_093
bg_004	bg_014	bg_024	bg_034	bg_044	bg_054	bg_064	bg_074	bg_084	bg_094
bg_005	bg_015	bg_025	bg_035	bg_045	bg_055	bg_065	bg_075	bg_085	bg_095
bg_006	bg_016	bg_026	bg_036	bg_046	bg_056	bg_066	bg_076	bg_086	bg_096
bg_007	bg_017	bg_027	bg_037	bg_047	bg_057	bg_067	bg_077	bg_087	bg_097
bg_008	bg_018	bg_028	bg_038	bg_048	bg_058	bg_068	bg_078	bg_088	bg_098
bg_009	bg_019	bg_029	bg_039	bg_049	bg_059	bg_069	bg_079	bg_089	bg_099


## Unzip the Entire DATASET

This will take a long time, ~3-4 hours, instead use the partial dataset and then create own

In [0]:
!unzip -n depth_dataset_cleaned.zip

## Unzip Partial, Create the Dataset later

In [0]:
!unzip depth_dataset_cleaned_raw.zip -d depth_dataset_cleaned/

# Create the Dataset

In [0]:
import glob
import PIL
from PIL import Image
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from pathlib import Path
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!ls depth_dataset_cleaned/

bg  fg	fg_bg  fg_bg_mask  fg_mask  labels


In [0]:
fgc_images = [f for f in glob.glob('depth_dataset_cleaned/fg/*.*')]
bgc_images = [f for f in glob.glob('depth_dataset_cleaned/bg/*.*')]
fgc_mask_images = [f for f in glob.glob('depth_dataset_cleaned/fg_mask/*.*')]

In [0]:
last_idx = 15

This was my attempt on creating the files direcctly into folders, this does work, however is a tedious process

In [0]:
idx = 0
for bidx, bg_image in enumerate(tqdm(bgc_images)):

    if (bidx < last_idx):
        continue

    Path(f'depth_dataset_cleaned/labels/').mkdir(parents=True, exist_ok=True)
    label_info = open(f"depth_dataset_cleaned/labels/bg_{bidx:03d}_label_info.txt","w+")

    idx = 4000 * bidx

    print(f'Processing BG {bidx}')
    Path(f'depth_dataset_cleaned/fg_bg/bg_{bidx:03d}').mkdir(parents=True, exist_ok=True)
    Path(f'depth_dataset_cleaned/fg_bg_mask/bg_{bidx:03d}').mkdir(parents=True, exist_ok=True)

    for fidx, fg_image in enumerate(tqdm(fgc_images)):
        # do the add fg to bg 20 times
        for i in range(20):
            # do this twice, one with flip once without
            for should_flip in [True, False]:
                background = Image.open(bg_image)
                foreground = Image.open(fg_image)
                fg_mask = Image.open(fgc_mask_images[fidx])
                
                if should_flip:
                    foreground = foreground.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                    fg_mask = fg_mask.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                    
                b_width, b_height = background.size
                f_width, f_height = foreground.size
                max_y = b_height - f_height
                max_x = b_width - f_width
                pos_x = np.random.randint(low=0, high=max_x, size=1)[0]
                pos_y = np.random.randint(low=0, high=max_y, size=1)[0]
                background.paste(foreground, (pos_x, pos_y), foreground)
                
                mask_bg = Image.new('L', background.size)

                fg_mask = fg_mask.convert('L')
                mask_bg.paste(fg_mask, (pos_x, pos_y), fg_mask)

                background.save(f'depth_dataset_cleaned/fg_bg/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_{idx:06d}.jpg', optimize=True, quality=30)
                mask_bg.save(f'depth_dataset_cleaned/fg_bg_mask/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_mask_{idx:06d}.jpg', optimize=True, quality=30)
                label_info.write(f'fg_bg/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_{idx:06d}.jpg\tfg_bg_mask/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_mask_{idx:06d}.jpg\t{pos_x}\t{pos_y}\n')
                
                idx = idx + 1

    label_info.close()
    last_idx = bidx


Processing BG 15



Processing BG 16



Processing BG 17



Processing BG 18



Processing BG 19


...


Processing BG 95



Processing BG 96



Processing BG 97



Processing BG 98



Processing BG 99


## This is how i created the .zip file
the output is removed, because the actual run was made on local machine

In [0]:
idx = 0
# for each background image
for bidx, bg_image in enumerate(tqdm(bgc_images)):
    # output zip file, open in append mode
    out_zip = ZipFile('fg_bg.zip', mode='a', compression=zipfile.ZIP_STORED)

    # labels for the craeted images
    label_info = open(f't_label.txt', 'w+')

    idx = 4000 * bidx

    print(f'Processing BG {bidx}')

    for fidx, fg_image in enumerate(tqdm(fgc_images)):
        # do the add fg to bg 20 times
        for i in range(20):
            # do this twice, one with flip once without
            for should_flip in [True, False]:
                # open the bg and fg images
                background = Image.open(bg_image)
                foreground = Image.open(fg_image)
                fg_mask = Image.open(fgc_mask_images[fidx])
                
                # if the fg image should be flipped
                if should_flip:
                    foreground = foreground.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                    fg_mask = fg_mask.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                
                # choose a random point on the bg to paste the fg image
                b_width, b_height = background.size
                f_width, f_height = foreground.size
                max_y = b_height - f_height
                max_x = b_width - f_width
                pos_x = np.random.randint(low=0, high=max_x, size=1)[0]
                pos_y = np.random.randint(low=0, high=max_y, size=1)[0]
                background.paste(foreground, (pos_x, pos_y), foreground)
                
                mask_bg = Image.new('L', background.size)

                fg_mask = fg_mask.convert('L')
                mask_bg.paste(fg_mask, (pos_x, pos_y), fg_mask)

                label_info.write(f'fg_bg/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_{idx:06d}.jpg\fg_bg_mask/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_mask_{idx:06d}.jpg\t{pos_x}\t{pos_y}\n')

                # save the background and the mask as temp .jpg files
                background.save('b_temp.jpg', optimize=True, quality=30)
                mask_bg.save('m_temp.jpg', optimize=True, quality=30)

                # save the files to .zip file
                out_zip.write('b_temp.jpg', f'depth_dataset_cleaned/fg_bg/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_{idx:06d}.jpg')
                out_zip.write('m_temp.jpg', f'depth_dataset_cleaned/fg_bg_mask/bg_{bidx:03d}/fg_{fidx:03d}_bg_{bidx:03d}_mask_{idx:06d}.jpg')
                
                idx = idx + 1

    label_info.close()

    # write the labels file to zip
    out_zip.write('t_labels.txt', f'depth_dataset_cleaned/labels/bg_{bidx:03d}_label_info.txt')

    # important: close the zip, else it gets corrupted
    out_zip.close()